# MODNet 'matbench_log_gvrh' and 'matbench_log_kvrh' benchmarking

Multi-target benchmarking of the ~11,000 entry elastic properties datasets provided by matminer.

In [1]:
from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from matminer.datasets import load_dataset
from pymatgen.core import Composition

from modnet.preprocessing import MODData
from modnet.models import MODNetModel
from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



## Data exploration

In [10]:
df_gvrh = load_dataset("matbench_log_gvrh")
df_kvrh = load_dataset("matbench_log_kvrh")

Fetching matbench_log_kvrh.json.gz from https://ml.materialsproject.org/projects/matbench_log_kvrh.json.gz to /home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/datasets/matbench_log_kvrh.json.gz


In [25]:
df_combined = pd.DataFrame(df_gvrh)
df_combined["log10(K_VRH)"] = df_kvrh["log10(K_VRH)"]

In [26]:
df_combined

,structure,log10(G_VRH),log10(K_VRH)
0,"[[0. 0. 0.] Ca, [1.37728887 1.57871271 3.73949...",1.447158,1.707570
1,"[[3.14048493 1.09300401 1.64101398] Mg, [0.625...",1.518514,1.633468
2,"[[ 2.06884519 2.40627241 -0.45891585] Si, [1....",1.740363,1.908485
3,"[[2.06428082 0. 2.06428082] Pd, [0. ...",1.707570,2.117271
4,"[[3.09635262 1.0689416 1.53602403] Mg, [0.593...",1.602060,1.690196
...,...,...,...
10982,"[[0. 0. 0.] Rh, [3.2029368 3.2029368 2.09459...",1.414973,1.778151
10983,"[[-1.51157821 4.4173925 1.21553922] Mg, [3....",1.431364,1.724276
10984,"[[4.37546772 4.51128393 6.81784473] H, [0.4573...",1.000000,1.342423
10985,"[[0. 0. 0.] Si, [ 4.55195829 4.55195829 -4.55...",1.579784,1.770852


### Target space

In [28]:
df_combined.describe()

,log10(G_VRH),log10(K_VRH)
count,10987.000000,10987.000000
mean,1.553639,1.880446
std,0.371615,0.369331
min,0.000000,0.000000
25%,1.342423,1.672098
50%,1.579784,1.934498
75%,1.826075,2.158362
max,2.718502,2.759668


In [ ]:
fig, ax = plt.subplots(facecolor="w")
ax.hist(df_combined["log10(G_VRH)"], bins=100, density=True, hist_type="step");
ax.set_ylabel("Frequency")
ax.set_xlabel("log10(G_VRH)")
fig, ax = plt.subplots(facecolor="w")
ax.hist(df_combined["log10(K_VRH)"], bins=100, density=True);
ax.set_ylabel("Frequency")
ax.set_xlabel("log10(K_VRH)")
fig, ax = plt.subplots(facecolor="w")
ax.scatter(df_combined["log10(G_VRH)"], df_combined["log10(K_VRH)"], alpha=0.1)
ax.set_xlabel("log10(G_VRH)")
ax.set_ylabel("log10(K_VRH)")

## Featurization and feature selection

In [40]:
data.save(PRECOMPUTED_MODDATA)

INFO:root:Data successfully saved as ./precomputed/elastic_benchmark_moddata.pkl.gz!


In [39]:
PRECOMPUTED_MODDATA = "./precomputed/elastic_benchmark_moddata.pkl.gz"

if os.path.isfile(PRECOMPUTED_MODDATA):
    data = MODData.load(PRECOMPUTED_MODDATA)
else:
    data = MODData(
        structures=df_combined["structure"].tolist(), 
        targets=df_combined[["log10(G_VRH)", "log10(K_VRH)"]],
        target_names=["log10(G_VRH)", "log10(K_VRH)"],
        featurizer=DeBreuck2020Featurizer(n_jobs=8)
    )
    data.featurize()
    # As this is a small data/feature set, order all features 
    data.feature_selection(n=-1)

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Computing features, this can take time...
INFO:root:Applying composition featurizers...
INFO:root:Applying featurizers (AtomicOrbitals(), AtomicPackingEfficiency(), BandCenter(), ElementFraction(), ElementProperty(data_source=<matminer.utils.data.MagpieData object at 0x7f68cc7b9400>,
                features=['Number', 'MendeleevNumber', 'AtomicWeight',
                          'MeltingT', 'Column', 'Row', 'CovalentRadius',
                          'Electronegativity', 'NsValence', 'NpValence',
                          'NdValence', 'NfValence', 'NValence', 'NsUnfilled',
                          'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled',
                          'GSvolume_pa', 'GSbandgap', 'GSmagmom',
                          'SpaceGroupNumber'],
                stats=['minimum', 'maximum', 'range', 'mean', 'avg_dev',
                       'mode']), IonProperty(), Miedema(ss_types=['min'], struct_types=['in

/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1369: RuntimeWarning: invalid value encountered in double_scalars
  hab_elast = ((2 * compr[0] * shear_mod[1] *
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1374: RuntimeWarning: invalid value encountered in double_scalars
  hba_elast = ((2 * compr[1] * shear_mod[0] *
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in po

/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  

/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  

/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1357: RuntimeWarning: invalid value encountered in power
  alp_a = (np.multiply(1.5, np.power(v_a, 2 / 3)) /
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/matminer/featurizers/composition.py:1369: RuntimeWarning: invalid value encountered in double_s

...

/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sites = np.array(sites)[uniq_inds]
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sites = np.array(sites)[uniq_inds]
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-

/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sites = np.array(sites)[uniq_inds]
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sites = np.array(sites)[uniq_inds]
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-

/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sites = np.array(sites)[uniq_inds]
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sites = np.array(sites)[uniq_inds]
/home/modl/mevans/.conda/envs/modnet/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:805: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-

INFO:root:Data has successfully been featurized!
INFO:root:Computing "self" MI (i.e. information entropy) of features
INFO:root:Computing cross NMI between all features...
  0%|          | 9/4223 [2:57:22<1384:14:17, 1182.55s/it]


KeyboardInterrupt: 

In [49]:
data.save(PRECOMPUTED_MODDATA)

INFO:root:Data successfully saved as ./precomputed/elastic_benchmark_moddata.pkl.gz!


In [ ]:
data.feature_selection(n=-1)

INFO:root:Computing "self" MI (i.e. information entropy) of features
INFO:root:Computing cross NMI between all features...
  1%|▏         | 54/4223 [17:38:59<1128:01:53, 974.07s/it]

In [ ]:
data.cross_nmi

In [ ]:
!ls precomputed

## Training

This is a very small dataset, so we must make judicious use of what data there is. First, let's generate test folds according to [matbench's suggestions](https://hackingmaterials.lbl.gov/automatminer/datasets.html#benchmarking-and-reporting-your-algorithm):